In [38]:
import tensorflow as tf
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from sklearn.model_selection import train_test_split
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

In [44]:
# Load the class labels
class_labels = ['class 1', 'class 2', 'class 3', 'class 4', 'class 5']

In [45]:
# Data preprocessing
data_dir = '/content/drive/MyDrive/data'
all_images = []
all_labels = []


In [ ]:
img_height,img_width=300,300
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
 val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with a given size
plt.figure(figsize=(10, 10))

# Iterate through the first batch of images and labels
for images, labels in train_ds.take(1):
    for i in range(9):  # Changed the range to show 9 images (3x3 grid)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))  # Display the image

        # Check the label and set the title accordingly
        if labels[i] < 5:
            class_num = labels[i] + 1  # Convert the label to class number (1-based index)
        else:
            class_num = 5  # If the label is not in the range of classes, set it as the last class

        plt.title(f"class {class_num}")  # Display the class number in the title
        plt.axis("off")  # Remove axis from the plot

# Display the plot
plt.show()

In [20]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(5, activation=tf.nn.softmax)
])


In [ ]:

from tensorflow.keras.optimizers import Adam

# Define the optimizer with the specified learning rate
adam_optimizer = Adam(learning_rate=0.0001)

# Compile the model using the defined optimizer
model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Continue with the model training
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [36]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_and_preprocess_image(model, image_path):
    """
    Loads the image from the given path, preprocesses it according to the model's input shape,
    and returns the preprocessed image and its prediction.
    """
    # Load the image and convert it to a numpy array
    image = load_img(
        image_path,
        target_size=tuple(model.input_shape[1:3]),
        resolve_ndarray=True,  # Return the image as a numpy array directly
    )

    # Perform any additional preprocessing if required
    preprocessed_image = preprocess_if_needed(image, model)

    # Make predictions on the preprocessed image
    predictions = model.predict(preprocessed_image)

In [ ]:
# Predict on the resized image
prediction = predict_on_image(model, '/content/drive/MyDrive/path/to/your/image.jpg')
print(f'Prediction: {prediction}')